# Import the google drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


### Copy the data and code to /content - gdrive

In [9]:
# ! cp -r /gdrive/MyDrive/img-quality-assesment/nima/ /content/
# ! cp -r /gdrive/MyDrive/nima-dataset/ /content/
# ! unzip /content/nima-dataset/nima-dataset.zip -d /content/nima-dataset
! rm /content/nima-dataset/nima-dataset.zip

In [23]:
# sync model to local folder
from shutil import copy2
def  sync_model_to_drive(model_file):
    dest_dir = '/gdrive/MyDrive/img-quality-assesment/nima/weights/'
    copy2(model_file, dest_dir)

DATASET_DIR = '/content/nima-dataset/content/nima-dataset/'
TID_DATASET = os.path.join(DATASET_DIR, 'tid2013')
TID_IMAGES = os.path.join(TID_DATASET,'distorted_images')
AVA_DATASET = os.path.join(DATASET_DIR, 'ava')
AVA_IMAGES = os.path.join(AVA_DATASET,'images')


In [7]:
! ls nima-dataset/

dataset-metadata.json  nima-dataset.zip


# Main code beigins here 
### Import the libraries

In [13]:
! pip install -q livelossplot

In [14]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os

import nima as nima
from nima.utils.ava_dataset_utils import load_data, get_rating_columns
from nima.config import INPUT_SHAPE, DATASET_DIR, CROP_SHAPE, MODEL_BUILD_TYPE, print_msg, WEIGHTS_DIR, TID_DATASET_DIR
from nima.model.data_generator import TrainDataGenerator, TestDataGenerator
from nima.model.loss import earth_movers_distance
from nima.model.model_builder import NIMA, model_weight_name

### Train the model on TID data

In [24]:
p_model_name = 'inception'
p_dataset_dir = AVA_DATASET
p_sample_size = None
p_weight_path = WEIGHTS_DIR
p_batch_size=128
p_metrics=['accuracy']
p_epochs=25
p_verbose=1

In [25]:
ava_dataset_dir = p_dataset_dir #os.path.join(p_dataset_dir, 'AVA')
ava_images_dir = os.path.join(ava_dataset_dir, 'images')
img_format = 'jpg'
print_msg(f'Images directory {ava_images_dir}')

# Load the dataset
x_col, y_cols = 'image_id', get_rating_columns()
df_train, df_valid, df_test = load_data(p_dataset_dir, p_sample_size)
assert len(df_train) > 0 and len(df_valid) > 0 and len(df_test) > 0, 'Empty dataframe'
train_batch_size = valid_batch_size = p_batch_size
test_batch_size = min(p_batch_size, 32, len(df_test))

# Form the NIMA Aesthetic Model
nima_aesthetic_cnn = NIMA(base_model_name=p_model_name, model_weights=p_weight_path,
                            loss=earth_movers_distance, input_shape=INPUT_SHAPE, metrics=p_metrics)

# Build the model for training
nima_aesthetic_cnn.build()
nima_aesthetic_cnn.compile()
nima_aesthetic_cnn.model.summary()

Images directory /content/nima-dataset/content/nima-dataset/ava/images
	Number of samples picked 176221
	NIMA Base CNN module - tensorflow.keras.applications.inception_v3.InceptionV3
87916544/87910968 [==============================] - 1s 0us/step
	Model compiled successfully.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
______________

In [26]:
# Get the generator
train_generator = TrainDataGenerator(df_train, ava_images_dir, x_col=x_col, y_col=y_cols,
                                        img_format=img_format, num_classes=10,
                                        preprocess_input=nima_aesthetic_cnn.get_preprocess_function(),
                                        batch_size=train_batch_size, input_size=INPUT_SHAPE, crop_size=CROP_SHAPE)
valid_generator = TrainDataGenerator(df_valid, ava_images_dir, x_col=x_col, y_col=y_cols,
                                        img_format=img_format, num_classes=10,
                                        preprocess_input=nima_aesthetic_cnn.get_preprocess_function(),
                                        batch_size=train_batch_size, input_size=INPUT_SHAPE, crop_size=CROP_SHAPE)



	Found 133927 valid image filenames belonging to 10 classes.
	Found 33482 valid image filenames belonging to 10 classes.


In [ ]:
    # Train the model
    print_msg("Training Aesthetic Model...")
    train_result_df, train_weights_file = nima_aesthetic_cnn.train_model(train_generator, valid_generator,
                                                                         epochs=p_epochs, verbose=p_verbose)
